# 1. SAT
#### Board setup
- Use $N^2$ variables: $x_{ij}, \forall i,j$ in range `[0,N-1]` 
- $x_{ij} = 1$ iff (if and only if) a queen is placed into the cell (i, j)


#### Constraints
- Each row must have 1 queen: For each $0 \leq i < N$, the i-th row contains at least one queen
    + For each $0 \leq i < N$: $(x_{i1} = 1$ or $x_{i2} = 1$ or $\dots$ or $x_{i(N−1)} = 1)$
- Row constraints: For each $0 \leq i < N$, the i-th row contains at most one queen
    + for each $0 \leq j_1 \neq j_2 < N$: $(x_{ij_1} = 0$ or $x_{ij_2} = 0)$

- Column constraints: For each $0 \leq j < N$, the j-th col contains at most one queen
    + for each $0 \leq i_ 1 \neq i_2 < N$: $(x_{i_1j} = 0$ or $x_{i_2j} = 0)$

- Diag constraints: no two queens stay on the same diagonal
    + For each pair of cell $(i_1, j_1)$ and $(i_2, j_2)$, $i_1 \neq i_2$  if $|i_1 − i_2 | == | j_1 − j_2 |$: $(x_{i_1j_1} = 0$ or $x_{i_2j_2} = 0)$
        + $|i_1 − i_2 | == | j_1 − j_2 |$ Simplified from: $(i_1 - j_1 == i_2 - j_2)$ or $(i_1 + j_1 == i_2 + j_2)$ 
        
| <img src="./img/3.jpg" width="500"/> | <img src="./img/4.jpg" width="650"/> |
|--------------------------------------|--------------------------------------|

In [1]:
from itertools import combinations, product
import pycosat


class N_Queen_SAT(object):
    def __init__(self):
        self._N = None

    def __get_var_id(self, r, c):
        '''Convert (r,c) in range [0, N) to var id in range [1, N^2]'''
        assert 0 <= r and r < self._N and \
            0 <= c and c < self._N
        return r*self._N + c + 1

    def solve(self, N):
        self._N = N

        ## Add constraints
        clauses = []

        # Constraint: Each row contains at least one queen
        for r in range(self._N):
            clauses.append([self.__get_var_id(r, c) for c in range(self._N)])

        # Constraint: Each row contains at most one queen
        for r in range(self._N):
            for c1, c2 in combinations(range(self._N), 2):
                clauses.append( [-self.__get_var_id(r, c1), -self.__get_var_id(r, c2)] )

        # Constraint: Each column contains at most one queen
        for c in range(self._N):
            for r1, r2 in combinations(range(self._N), 2):
                clauses.append( [-self.__get_var_id(r1, c), -self.__get_var_id(r2, c)] )

        # Diag constraints: no two queens stay on the same diagonal
                # if same slash / diag (r1 + c1 == r2 + c2)
                #    or backslash \ diag (r1 - c1 + N-1 == r1 - c1 + N-1)
                # Simplified to |r1 - r2| == |c1 - c2| if Q1, Q2 on same diag
        for r1, c1, r2, c2 in product(range(self._N), repeat=4):
            if (r1 != r2 or c1 != c2) \
                    and abs(r1 - r2) == abs(c1 - c2):
                clauses.append( [-self.__get_var_id(r1, c1), -self.__get_var_id(r2, c2)] )

        # Solve SAT
        assignment = pycosat.solve(clauses)
        
        # Print ans
        for r in range(self._N):
            for c in range(self._N):
                val = 1 if assignment[self.__get_var_id(r, c) - 1] > 0 else 0
                end_c = '\n' if c == self._N - 1 else ' '
                print(f'{val}', end=end_c)

In [2]:
%%time
sat = N_Queen_SAT()
sat.solve(N=10)

0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 1 0 0
0 0 0 0 1 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 1 0 0 0 0
0 0 1 0 0 0 0 0 0 0
CPU times: user 6.17 ms, sys: 111 µs, total: 6.28 ms
Wall time: 5.53 ms


# 2. Linear Programming - Pulp

In [3]:
from pulp import LpMaximize, LpProblem, LpStatus, LpVariable, PULP_CBC_CMD


class N_Queen_LP(object):
    def __init__(self):
        self._N = None

    def __get_var_id(self, r, c):
        '''Convert (r,c) in range [0, N) to var id in range [0, N^2)'''
        assert 0 <= r and r < self._N and \
            0 <= c and c < self._N
        return r*self._N + c

    def solve(self, N):
        self._N = N

        # Define the model
        model = LpProblem(name="N_Queen", sense=LpMaximize)

        # Define the decision variables
        x = {self.__get_var_id(r, c): LpVariable(name=f"board[{r}][{c}]", cat="Binary") \
            for r in range(self._N) for c in range(self._N)}

        # Constraint: Each row contains at least/most 1 Queen
        for r in range(self._N):
            queen_per_row = 0
            for c in range(self._N):
                queen_per_row += x[self.__get_var_id(r, c)]
            model += (queen_per_row <= 1, f"row {r} contains at most 1 Queen")

        # Constraint: Each col contains at most 1 Queen
        for c in range(self._N):
            queen_per_col = 0
            for r in range(self._N):
                queen_per_col += x[self.__get_var_id(r, c)]
            model += (queen_per_col <= 1, f"Each col {c} contains at most 1 Queen")

        # Constraint: Each slash \ diagonal contains at most 1 Queen
        slash = {i: 0 for i in range(2*self._N - 1)}
        for r in range(self._N):
            for c in range(self._N):
                slash[r+c] += x[self.__get_var_id(r, c)]
        for i in range(2*self._N - 1):
            model += (slash[i] <= 1, f"Each slash {i} contains at most 1 Queen")

        # Constraint: Each backslash / diagonal contains at most 1 Queen
        backslash = {i: 0 for i in range(2*self._N - 1)}
        for r in range(self._N):
            for c in range(self._N):
                backslash[r - c + self._N - 1] += x[self.__get_var_id(r, c)]
        for i in range(2*self._N - 1):
            model += (backslash[i] <= 1, f"Each backslash {i} contains at most 1 Queen")

        # Constraint: Total Queen <= N
        total_queen = 0
        for r in range(self._N):
            for c in range(self._N):
                total_queen += x[self.__get_var_id(r, c)]
        model += (total_queen <= self._N, f"Total Queen <= {self._N}")

        # Ojective: Max num of Queens
        model += total_queen

        # Solve the optimization problem
        solver = PULP_CBC_CMD()
        status = model.solve(PULP_CBC_CMD(msg=False))

        # Print ans
        for r in range(self._N):
            for c in range(self._N):
                val = str(int(x[self.__get_var_id(r,c)].value()))
                end_c = '\n' if c == self._N - 1 else ' '
                print(f'{val}', end=end_c)

In [4]:
%%time
lp = N_Queen_LP()
lp.solve(N=20)

0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
CPU times: user 23.1 ms, sys: 5.58 ms, total: 28.7 ms
Wall time: 64.3 ms
